In [1]:
import json
from matchms.importing import load_from_mgf
from rdkit import Chem
# from massspecgym.tools.analyzers import analyze_canonical_smiles
import os

In [2]:
import importlib
import massspecgym.data.transforms as transforms
import massspecgym.data.datasets as datasets
import massspecgym.data.data_module as data_module




In [93]:
from massspecgym.data.transforms import MolFingerprinter, MolToInChIKey, MolToFormulaVector, SpecTokenizer
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
from massspecgym.data.data_module import MassSpecDataModule
import torch

In [4]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [5]:
from massspecgym.data.transforms import MolFingerprinter, MolToInChIKey, MolToFormulaVector, SpecTokenizer
# from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset
# from massspecgym.featurize import SpectrumFeaturizer
# from massspecgym.data.data_module import MassSpecDataModule
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [6]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [7]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [8]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [9]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [10]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

I am not here
{'spec': tensor([[[1.6708e+02, 1.1000e+00],
         [6.3188e+01, 2.2353e-04],
         [7.7039e+01, 5.0969e-04],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[4.5414e+02, 1.1000e+00],
         [5.8257e+01, 1.1829e-03],
         [6.5189e+01, 1.2319e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[2.0309e+02, 1.1000e+00],
         [5.7045e+01, 1.3727e-03],
         [7.2045e+01, 1.6131e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        ...,

        [[2.6915e+02, 1.1000e+00],
         [7.9054e+01, 4.9049e-02],
         [8.1070e+01, 1.9019e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.1512e+02, 1.1000e+00],
         [1.0503e+02, 5.

In [12]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 223, 256, 256, 256, 256, 256, 256, 256, 256])


In [13]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [14]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [15]:
tmp_original['precursor_mz']

tensor([167.0820, 454.1420, 203.0927,  96.0449, 596.1974, 259.0077, 271.0965,
        340.2100, 254.1288, 269.1536, 315.1227, 470.0339])

In [16]:
tmp_original['adduct'], 

(['[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+'],)

In [17]:
tmp_original['mol_freq']

tensor([ 2.,  2., 65., 10.,  7., 32., 11., 25.,  7., 21., 13., 21.])

In [18]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [90]:
sum(tmp_original['labels'])

tensor(12)

In [92]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768,  991, 1247, 1503, 1759, 2015, 2271, 2527, 2783])

In [19]:
len(tmp_original['labels'])

3039

In [20]:
tmp_original['batch_ptr']

tensor([256, 256, 256, 223, 256, 256, 256, 256, 256, 256, 256, 256])

In [21]:
sum(tmp_original['batch_ptr'])

tensor(3039)

In [22]:
len(tmp_original['candidates'])

3039

In [23]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [24]:
len(tmp_original['candidates_smiles'])

3039

# MSnRetrieval

In [25]:
from massspecgym.featurize import SpectrumFeaturizer

In [26]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [27]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [28]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [30]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 15674
Dataset length: 15674


In [31]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [32]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 11883
Val dataset size: 1899


In [33]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

in collate_fn
{'spec': DataBatch(x=[154, 1039], edge_index=[2, 142], batch=[154], ptr=[13]), 'mol': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'precursor_mz': tensor([595.1658, 397.1922, 330.1812, 540.2428, 316.1768, 399.1490, 457.1983,
        331.0999, 466.2714, 304.1180, 355.1401, 450.2387]), 'adduct': ['[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+NH4]+', '[M+H]+', '[M+H]+'], 'identifier': ['0005049_0000000', '0071393_0000000', '0045358_0000000', '0062206_0000000', '0058719_0000000', '0082987_0000000', '0044383_0000000', '0019701_0000000', '0079814_0000000', '0005715_0000000', '0030518_0000000', '0011550_0000000'], 'mol_freq': tensor([1., 3., 1., 1., 1., 1., 1., 1., 1., 3., 1., 1.]), 'smiles': ['C[C@H]1[C@@H]([C@H]([C@H]([C@@H

In [34]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [35]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [36]:
tmp_msn['spec'] 

DataBatch(x=[154, 1039], edge_index=[2, 142], batch=[154], ptr=[13])

In [37]:
tmp_msn['precursor_mz']

tensor([595.1658, 397.1922, 330.1812, 540.2428, 316.1768, 399.1490, 457.1983,
        331.0999, 466.2714, 304.1180, 355.1401, 450.2387])

In [38]:
tmp_msn['mol_freq']

tensor([1., 3., 1., 1., 1., 1., 1., 1., 1., 3., 1., 1.])

In [39]:
tmp_msn['identifier']

['0005049_0000000',
 '0071393_0000000',
 '0045358_0000000',
 '0062206_0000000',
 '0058719_0000000',
 '0082987_0000000',
 '0044383_0000000',
 '0019701_0000000',
 '0079814_0000000',
 '0005715_0000000',
 '0030518_0000000',
 '0011550_0000000']

In [40]:
tmp_msn['adduct']

['[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+NH4]+',
 '[M+H]+',
 '[M+H]+']

In [41]:
tmp_msn['mol'].shape

torch.Size([12, 4096])

In [42]:
tmp_msn['smiles']

['C[C@H]1[C@@H]([C@H]([C@H]([C@@H](O1)OC2=CC(=C3C(=C2)OC(=C(C3=O)O[C@H]4[C@@H]([C@H]([C@@H]([C@H](O4)CO)O)O)O)C5=CC=C(C=C5)O)O)O)O)O',
 'CC(C1=CC2=C(C=C1)N(CC2)C(=O)C3CCOCC3)NC(=O)C4=CC=C(C=C4)F',
 'C1CC(=O)NC(=O)C1N2CC3=C(C=CC=C3C2=O)CCCCCN',
 'CCN(CC)CCNC(=O)C1=C(NC(=C1C)/C=C\\2/C3=C(C=CC(=C3)C4=CSC(=N4)C5=CC=CC=C5)NC2=O)C',
 'COC1=CC2=NNN=C2C=C1C(=O)NCC3CCCN3CC=C',
 'C1=CC(=CC(=C1)OCC(C(F)F)(F)F)CNCCC2=CNC3=C2C=CC(=C3)F',
 'CCC1=CC=CC=C1NC(=O)CN2C(=O)N3C4=C(C(=O)N(C3=N2)C)N(C=C4C5=CC=CC=C5)C',
 'CCOC(=O)C(C)(C)OC1=CC=CC2=C1S(=O)C3=CC=CC=C23',
 'CC1=C(C=CC(=C1)N(C)C)C2=NC3=C(N2)C=C(C=C3)C4=NC5=C(N4)C=C(C=C5)N6CCN(CC6)C',
 'C/C(=C/COC1=C2C(=CC3=C1OC=C3)C=CC(=O)O2)/CO',
 'CC1=CC(=C(C(=O)N1)C(=O)NCCN2C3=CC=CC=C3NC(=O)C2=O)C',
 'CC1(C=CC2=C(O1)C3=C(C=C2)C4(CC5C(=O)N6CCCC6C(=O)N5C4(N3)C(C)(C)C=C)O)C']

In [43]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [94]:
sum(tmp_msn['labels'])

tensor(12)

In [95]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1280, 1536, 1792, 2048, 2304, 2560, 2816])

In [44]:
len(tmp_msn['labels'])

3072

In [45]:
tmp_msn['batch_ptr']

tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])

In [46]:
sum(tmp_msn['batch_ptr'])

tensor(3072)

In [47]:
len(tmp_msn['candidates'])

3072

In [48]:
tmp_msn['candidates'][5].shape

torch.Size([4096])

### Compare Shapes

In [49]:
tmp_msn.keys() == tmp_original.keys()

True

In [50]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [54]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [64]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

True

In [87]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(3072), 3072)

In [88]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(3039), 3039)

In [70]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [72]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [75]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [78]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [80]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [79]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [82]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [83]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [86]:
len(tmp_msn['candidates_smiles'])

3072